In [6]:
import numpy as np
import pandas as pd
import datatable
import itertools as it
from owlready2 import *
import math
import joblib

import pickle
from bayes_opt import BayesianOptimization
#para depurar los que letras que no están en el codigo ascii
import unicodedata
import functools
import spacy
import stanfordnlp
#from spacy_stanfordnlp import StanfordNLPLanguage
from sklearn.decomposition import LatentDirichletAllocation
from sklearn import metrics
#from wiki_dump_reader import Cleaner, iterate
from sklearn.decomposition import TruncatedSVD

from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import pairwise_distances

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import chi2
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import normalize,scale



#import pyLDAvis
#import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

from IPython.display import SVG

import random
from sklearn.externals import joblib


#Operaciones con gráficos
import scattertext as st
from matplotlib import pyplot as plt
from matplotlib.ticker import MaxNLocator

import networkx as nx

import bokeh.plotting as bp
from bokeh.models import HoverTool, BoxSelectTool, LabelSet, ColumnDataSource, Range1d
from bokeh.plotting import figure, show, output_notebook
from sklearn.manifold import TSNE

#import gensim
#import gensim.corpora as corpora
#from gensim.models import CoherenceModel
#from gensim.models import HdpModel

warnings.filterwarnings("ignore")

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from multiprocessing import  Pool
import math
import scipy.sparse as sp


from sklearn.model_selection import *
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix



In [2]:
import json

with open('../config/config.json', 'r') as f:
    config = json.load(f)


In [3]:
interimpath = '../data/interim/reuters/corpus_normalizado.csv'

In [4]:
interimpath

'../data/interim/reuters/corpus_normalizado.csv'

In [5]:
corpus = pd.read_csv(interimpath)

corpus = corpus.loc[:,['texto_normalizado','Target']]
corpus.Target = corpus.Target.astype(int)
corpus.head()

,texto_normalizado,Target
0,asian exporter fear damage u.s.-japan rift mou...,0
1,china daily say vermin eat 7 12 pct grain stoc...,0
2,japan revise long term energy demand downwards...,0
3,thai trade deficit widen quarter thailand trad...,0
4,indonesia see cpo price rise sharply indonesia...,0


In [ ]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')


In [ ]:
stop_words.append('be')
stop_words.append('don')
from sklearn.feature_extraction import text
my_stop_words = text.ENGLISH_STOP_WORDS.union(["be","don"])


In [ ]:
random.seed(100)

Para hacer LDA, no vamos a utilizar todo el corpus. Seleccionaremos aleatoriamente 100.000 textos de clase positiva y sin clasificar para posteriormente hacer LDA.

In [198]:
 tfidf = TfidfVectorizer(min_df=0.025,max_df=0.95, stop_words=['english','spanish'], ngram_range=(1,1))
#por ahora no pongo ngramas
joblib.dump(tfidf,'../objects/reuters/tfidf_object')

['../objects/reuters/tfidf_object']

In [199]:
dtm = tfidf.fit_transform(corpus.iloc[:,0])

In [200]:
dtm
joblib.dump(dtm,'../objects/reuters/dtm.final')

['../objects/reuters/dtm.final']

### Anteriormente, hemos realizado un grid search. Vamos a utilizar los parámetros que mejor funcionaron en su momento

from sklearn.model_selection import GridSearchCV
search_params = { 'topic_word_prior':[0.5,0.7,0,0.95,0.99], # hemos partido de 0.5, 0.7, 0,9, 0.95 y 0.99
                 'doc_topic_prior':[0.01,0.015,0.05,0.3,0.5]}

LDA = LatentDirichletAllocation(n_components=2,random_state=42,n_jobs=10,batch_size=50000,  evaluate_every = 100,  mean_change_tol=0.05)

model = GridSearchCV(LDA, param_grid=search_params)
from sklearn.utils import parallel_backend
i = time.time()
with parallel_backend('threading'):
    # Do the Grid Search
    model.fit(dtm)

    joblib.dump(model, 'lda_model4.pkl')

f = time.time()
elapsed = f - i
print('transcurridos {0} s'.format(elapsed))


## Ya sabemos que nuestro mejor modelo tiene unos priors determinados, 
joblib.dump(modelo,'../objects/'+config.get('corpus')+'/modelo_gridsearch.final')


In [ ]:
with parallel_backend('threading'):
    LDA = LatentDirichletAllocation(n_components=2,random_state=42,topic_word_prior = 0.95,doc_topic_prior = 0.05,n_jobs=10,max_iter=250,mean_change_tol=0.05)

    joblib.dump(LDA,'../objects/reuters/lda.final')


In [ ]:
#tfidf = joblib.load('tfidf')
tfidf

In [ ]:
texts = corpus.iloc[:,0]
texts = list(texts.values)

In [ ]:
feature_names = tfidf.get_feature_names()

In [ ]:
feature_names

In [ ]:
i = time.time()
from sklearn.utils import parallel_backend
i = time.time()
with parallel_backend('threading'):
    modelo = LDA.fit(dtm)
print (' terminado en {0} min'.format((time.time() - i) / 60))

In [ ]:
joblib.dump(modelo,'../objects/reuters/final_lda_fit.final.pck')
joblib.dump(feature_names,'../objects/reuters/feature_names.final')


In [33]:
#modelo = joblib.load('final_lda.pck')
#modelo = joblib.load('final_lda.final.pck')


In [6]:
 from sklearn.externals import joblib
modelo = joblib.load('../objects/reuters/final_lda_fit.final.pck')

In [7]:
tfidf = joblib.load('../objects/reuters/tfidf_object')

In [11]:
feature_names = joblib.load('../objects/reuters/feature_names.final')

In [14]:
for index,topic in enumerate(modelo.components_):
    print(f'THE TOP 15 WORDS FOR TOPIC #{index}')
    print([feature_names[i] for i in topic.argsort()[-15:]])
    joblib.dump([feature_names[i] for i in topic.argsort()],'../objects/reuters/topics'+str(index))
    print('\n')
    

THE TOP 15 WORDS FOR TOPIC #0
['market', 'trade', 'tonne', 'billion', 'oil', 'price', 'year', 'lt', 'bank', 'company', 'share', 'mln', 'dlrs', 'pct', 'say']


THE TOP 15 WORDS FOR TOPIC #1
['record', 'dividend', 'div', 'lt', 'dlrs', 'profit', 'rev', 'qtr', 'shr', 'loss', '000', 'net', 'mln', 'cts', 'vs']




In [ ]:
for index,topic in enumerate(modelo.components_):
    print(f'THE TOP 15 WORDS FOR TOPIC #{index}')
    print([tfidf.get_feature_names()[i] for i in topic.argsort()[-15:]])
    joblib.dump([tfidf.get_feature_names()[i] for i in topic.argsort()],'topics'+str(index))
    print('\n')

In [ ]:
#save dtm as dataframe

In [205]:
pds = pd.DataFrame(dtm.todense(), columns = feature_names)


In [206]:
joblib.dump(pds,'../objects/reuters/dtm_pds')

['../objects/reuters/dtm_pds']

In [207]:
pds = joblib.load('../objects/reuters/dtm_pds')
pds

,000,10,100,11,12,13,14,15,16,17,...,week,well,west,wheat,work,world,year,yen,yesterday,york
0,0.000000,0.02694,0.0,0.0,0.000000,0.000000,0.0,0.029077,0.0,0.033901,...,0.031049,0.000000,0.0,0.0,0.039837,0.072397,0.077299,0.000000,0.037801,0.0
1,0.000000,0.00000,0.0,0.0,0.262799,0.000000,0.0,0.000000,0.0,0.000000,...,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.082495,0.000000,0.000000,0.0
2,0.000000,0.00000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,...,0.000000,0.000000,0.0,0.0,0.072474,0.000000,0.070314,0.078192,0.000000,0.0
3,0.000000,0.00000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,...,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.089077,0.000000,0.000000,0.0
4,0.000000,0.00000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,...,0.000000,0.255582,0.0,0.0,0.000000,0.111554,0.059553,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10783,0.000000,0.00000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,...,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
10784,0.000000,0.00000,0.0,0.0,0.000000,0.228866,0.0,0.000000,0.0,0.000000,...,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
10785,0.000000,0.00000,0.0,0.0,0.415757,0.000000,0.0,0.000000,0.0,0.000000,...,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
10786,0.000000,0.00000,0.0,0.0,0.090106,0.000000,0.0,0.000000,0.0,0.000000,...,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.056570,0.000000,0.000000,0.0


In [208]:
ldares = modelo.transform(dtm)

In [209]:
joblib.dump(ldares,'../objects/reuters/ldares_transform.final')

['../objects/reuters/ldares_transform.final']

In [210]:
ldares = joblib.load('../objects/reuters/ldares_transform.final')

In [211]:
ldares

array([[0.99322099, 0.00677901],
       [0.98884685, 0.01115315],
       [0.99020117, 0.00979883],
       ...,
       [0.01323281, 0.98676719],
       [0.01518235, 0.98481765],
       [0.01440816, 0.98559184]])

In [212]:
data = pd.DataFrame(ldares,columns=['A','B'])
data['Target'] = corpus.iloc[:,1]

In [213]:
data.head()

,A,B,Target
0,0.993221,0.006779,0
1,0.988847,0.011153,0
2,0.990201,0.009799,0
3,0.990731,0.009269,0
4,0.991297,0.008703,0


In [214]:
datapos = data[data.Target==1]
dataneg = data[data.Target==0]

In [215]:
datapos.shape

(2369, 3)

In [216]:
dataneg.shape

(8419, 3)

In [217]:
datan  = datapos.append(dataneg.sample(n=3000))
datan.tail()

,A,B,Target
7006,0.991960,0.008040,0
8941,0.989182,0.010818,0
611,0.988334,0.011666,0
8183,0.983393,0.016607,0
10681,0.992039,0.007961,0


In [218]:
joblib.dump(corpus.iloc[:,1],'../objects/reuters/y_a_predecir')

['../objects/reuters/y_a_predecir']

In [219]:
##############


In [7]:
import h2o
from h2o.estimators import H2OXGBoostEstimator
h2o.init(max_mem_size='14G')


Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O cluster uptime:,3 hours 27 mins
H2O cluster timezone:,Europe/Berlin
H2O data parsing timezone:,UTC
H2O cluster version:,3.28.0.3
H2O cluster version age:,5 months and 10 days !!!
H2O cluster name:,H2O_from_python_rdelaguila_s6g6rh
H2O cluster total nodes:,1
H2O cluster free memory:,12.85 Gb
H2O cluster total cores:,12
H2O cluster allowed cores:,12
H2O cluster status:,"locked, healthy"


In [226]:
datan.to_csv('../objects/reuters/datan.delete.csv')

In [8]:
frame = h2o.upload_file('../objects/reuters/datan.delete.csv')

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [9]:
frame['Target']=frame['Target'].asfactor()

In [10]:
train,test=frame.split_frame(ratios=[.75])

In [235]:
def xgb_cv(max_depth, ntrees,min_rows,eta,learn_rate,sample_rate,colsample_bytree,reg_lambda,reg_alpha,train):
    """SVC cross validation.
    This function will instantiate a SVC classifier with parameters C and
    gamma. Combined with data and targets this will in turn be used to perform
    cross validation. The result of cross validation is returned.
    Our goal is to find combinations of C and gamma that maximizes the roc_auc
    metric.
    """
    params = {'max_depth': int(max_depth), 'ntrees': int(ntrees),'min_rows':int(min_rows),'eta':float(eta),
              'learn_rate':float(learn_rate),'sample_rate':float(sample_rate),
              'colsample_bytree':float(colsample_bytree),'reg_lambda':float(reg_lambda),'reg_alpha':float(reg_alpha)}
      
    estimator = H2OXGBoostEstimator(**params)
    estimator.train(x=['A','B'],y='Target',training_frame=train)
    return estimator.auc()

def optimize_xgb(train):
    """Apply Bayesian Optimization to XGB parameters."""
    def xgb_crossval(max_depth, ntrees,min_rows,eta,learn_rate,sample_rate,colsample_bytree,reg_lambda,reg_alpha,train=train):
        """Wrapper of SVC cross validation.
        Notice how we transform between regular and log scale. While this
        is not technically necessary, it greatly improves the performance
        of the optimizer.
        """
        return xgb_cv(max_depth, ntrees,min_rows,eta,learn_rate,sample_rate,colsample_bytree,reg_lambda,reg_alpha,train)

    optimizer = BayesianOptimization(
        f=xgb_crossval,
        
        pbounds={"max_depth": (4, 40), "ntrees": (50,400),'min_rows':(2,50),'eta':(0.01,1),
                 'learn_rate':(0.01,1),'sample_rate':(0.1,0.5),'colsample_bytree':(0.2,0.8),
                 'reg_lambda':(0,1),'reg_alpha':(0,1)},
        random_state=1234,
        verbose=2
    )
    optimizer.maximize(n_iter=2)

    print("Final result:", optimizer.max)
from bayes_opt.util import Colours
print(Colours.yellow("--- Optimizing SVM ---"))
train['A']= train['A'].asnumeric()
train['B']= train['B'].asnumeric()
optimize_xgb(train)

--- Optimizing SVM ---
|   iter    |  target   | colsam... |    eta    | learn_... | max_depth | min_rows  |  ntrees   | reg_alpha | reg_la... | sample... |
-------------------------------------------------------------------------------------------------------------------------------------
xgboost Model Build progress: |███████████████████████████████████████████| 100%
|  1        |  0.8355   |  0.3149   |  0.6259   |  0.4434   |  32.27    |  39.44    |  145.4    |  0.2765   |  0.8019   |  0.4833   |
xgboost Model Build progress: |███████████████████████████████████████████| 100%
|  2        |  0.698    |  0.7256   |  0.3642   |  0.506    |  28.6     |  36.21    |  179.6    |  0.5612   |  0.5031   |  0.1055   |
xgboost Model Build progress: |███████████████████████████████████████████| 100%
|  3        |  0.8975   |  0.6637   |  0.8838   |  0.3712   |  26.15    |  5.618    |  179.1    |  0.9331   |  0.6514   |  0.2589   |
xgboost Model Build progress: |█████████████████████████████████

In [237]:
params= {'colsample_bytree': 0.2, 'eta': 1.0, 'learn_rate': 0.01, 'max_depth': 4, 'min_rows': 2, 'ntrees': 400, 'reg_alpha': 1.0, 'reg_lambda': 0.0, 'sample_rate': 0.5}

train

In [239]:
model = H2OXGBoostEstimator(**params,seed=123)

In [242]:
model.train(x=['A','B'],y='Target',training_frame=train)

xgboost Model Build progress: |███████████████████████████████████████████| 100%


In [13]:
performance = model.model_performance(test)

In [14]:
performance


ModelMetricsBinomial: generic
** Reported on test data. **

MSE: 0.12258065454600184
RMSE: 0.35011520182077477
LogLoss: 0.39636583864456343
Mean Per-Class Error: 0.16198979591836737
AUC: 0.9033502072704082
AUCPR: 0.8409199308185897
Gini: 0.8067004145408163

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.40084634721279144: 


,,0,1,Error,Rate
0,0,608.0,160.0,0.2083,(160.0/768.0)
1,1,68.0,520.0,0.1156,(68.0/588.0)
2,Total,676.0,680.0,0.1681,(228.0/1356.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.400846,0.820189,247.0
1,max f2,0.216077,0.876062,317.0
2,max f0point5,0.641613,0.822419,161.0
3,max accuracy,0.456487,0.834808,225.0
4,max precision,0.921635,0.920530,40.0
5,max recall,0.002163,1.000000,399.0
6,max specificity,0.998120,0.998698,0.0
7,max absolute_mcc,0.400846,0.670041,247.0
8,max min_per_class_accuracy,0.483247,0.832031,218.0
9,max mean_per_class_accuracy,0.400846,0.838010,247.0


In [12]:
model = h2o.upload_mojo('../objects/reuters/modelo_xgb_val/XGBoost_model_python_1594477941824_1073.zip')

generic Model Build progress: |███████████████████████████████████████████| 100%
Model Details
H2OGenericEstimator :  Import MOJO Model
Model Key:  Generic_model_python_1594894675132_5816


Model Summary: 


,,number_of_trees
0,,400.0




ModelMetricsBinomialGeneric: generic
** Reported on train data. **

MSE: 0.07831380202697656
RMSE: 0.27984603271616443
LogLoss: 0.26196056557832975
Mean Per-Class Error: 0.09895838046942007
AUC: 0.9667533467878581
AUCPR: 0.9482246817506015
Gini: 0.9335066935757161

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.4287731647491455: 


,,0,1,Error,Rate
0,0,1938.0,298.0,0.1333,(298.0/2236.0)
1,1,115.0,1664.0,0.0646,(115.0/1779.0)
2,Total,2053.0,1962.0,0.1029,(413.0/4015.0)



Maximum Metrics: Maximum metrics at their respective thresholds


...

,,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain
0,,1,0.011208,0.986623,2.256886,2.256886,1.000000,0.992927,1.000000,0.992927,0.025295,0.025295,125.688589,125.688589
1,,2,0.021420,0.979723,2.201840,2.230643,0.975610,0.981713,0.988372,0.987581,0.022485,0.047780,120.183989,123.064303
2,,3,0.031133,0.974989,2.256886,2.238831,1.000000,0.976664,0.992000,0.984175,0.021922,0.069702,125.688589,123.883080
3,,4,0.040100,0.964389,2.256886,2.242868,1.000000,0.971133,0.993789,0.981259,0.020236,0.089938,125.688589,124.286797
4,,5,0.050311,0.957041,2.256886,2.245713,1.000000,0.960859,0.995050,0.977118,0.023047,0.112985,125.688589,124.571319
5,,6,0.100623,0.926696,2.234540,2.240127,0.990099,0.943081,0.992574,0.960100,0.112423,0.225408,123.454049,124.012684
6,,7,0.150436,0.899068,2.245601,2.241940,0.995000,0.913371,0.993377,0.944627,0.111861,0.337268,124.560146,124.193963
7,,8,0.200000,0.855811,2.211521,2.234401,0.979899,0.877980,0.990037,0.928110,0.109612,0.446880,121.152135,123.440135
8,,9,0.300125,0.746247,2.105304,2.191333,0.932836,0.804852,0.970954,0.886990,0.210793,0.657673,110.530400,119.133319
9,,10,0.400000,0.596648,1.688443,2.065767,0.748130,0.674353,0.915318,0.833897,0.168634,0.826307,68.844331,106.576728




Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_auc,training_pr_auc,training_lift,training_classification_error
0,,2020-07-11 16:36:56,0.004 sec,0.0,0.500000,0.693147,0.500000,0.000000,1.000000,0.556912
1,,2020-07-11 16:36:57,0.034 sec,1.0,0.435934,0.552603,0.734984,0.664509,1.887577,0.319054
2,,2020-07-11 16:36:57,0.047 sec,2.0,0.430598,0.530975,0.751409,0.641612,1.887577,0.319054
3,,2020-07-11 16:36:57,0.058 sec,3.0,0.427816,0.520821,0.764254,0.649827,2.056274,0.321046
4,,2020-07-11 16:36:57,0.071 sec,4.0,0.426810,0.518380,0.767089,0.654131,2.056274,0.312578
5,,2020-07-11 16:36:57,0.085 sec,5.0,0.425437,0.514700,0.770680,0.652974,2.056274,0.307098
6,,2020-07-11 16:36:57,0.097 sec,6.0,0.423921,0.511780,0.777027,0.671049,1.763192,0.304608
7,,2020-07-11 16:36:57,0.109 sec,7.0,0.422130,0.508098,0.781398,0.682467,1.853871,0.306600
8,,2020-07-11 16:36:57,0.121 sec,8.0,0.422358,0.508360,0.780852,0.687113,2.076335,0.304857
9,,2020-07-11 16:36:57,0.133 sec,9.0,0.422769,0.509528,0.780801,0.683963,1.907228,0.314321



See the whole table with table.as_data_frame()

Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,B,5289.568359,1.000000,0.503154
1,A,5223.262695,0.987465,0.496846


In [243]:
model.save_mojo('../objects/reuters/modelo_xgb_val')
h2o.shutdown()

H2O session _sid_93e0 closed.


# Analisis de anomalias
###aqui

## Añadimos probabilidad de pertenencia


In [6]:
import joblib
LDA = joblib.load('../objects/reuters/lda.final')
dtm = joblib.load('../objects/reuters/dtm.final')

In [7]:
topic_results = LDA.fit_transform(dtm)
corpus['Topic']=topic_results.argmax(axis=1)
corpus['prob_topic']=topic_results[:,1]

In [9]:
corpus.to_pickle('../data/processed/reuters/corpus.gzip',compression='gzip')

In [11]:
corpus.tail()

,texto_normalizado,Target,Topic,prob_topic
10783,u.k money market shortage forecast revise bank...,0,0,0.015244
10784,knight ridder inc lt;krn > set quarterly qtly ...,0,1,0.986222
10785,technitrol inc lt;tnl > set quarterly qtly div...,0,1,0.986767
10786,nationwide cellular service inc lt;ncel > 4th ...,0,1,0.984818
10787,lt;a.h.a automotive technology corp > year net...,0,1,0.985592


## IQR

In [12]:
q1, q3= np.percentile(corpus.loc[corpus.Topic==1,'prob_topic'],[25,75])

In [13]:
iqr = q3-q1
lower_bound = q1 -(1.5 * iqr) 
upper_bound = q3 +(1.5 * iqr) 

In [14]:
q1

0.9854422456425984

In [15]:
q3

0.988446344197184

In [16]:
iqr

0.0030040985545856103

In [17]:
lower_bound

0.98093609781072

In [18]:
upper_bound

0.9929524920290624

In [19]:
corpus.head()

,texto_normalizado,Target,Topic,prob_topic
0,asian exporter fear damage u.s.-japan rift mou...,0,0,0.006779
1,china daily say vermin eat 7 12 pct grain stoc...,0,0,0.011153
2,japan revise long term energy demand downwards...,0,0,0.009799
3,thai trade deficit widen quarter thailand trad...,0,0,0.009269
4,indonesia see cpo price rise sharply indonesia...,0,0,0.008703


In [20]:
corpus['outlier_iqr']=(corpus.Topic==1) & (corpus.prob_topic<lower_bound)

In [21]:
corpus.head()

,texto_normalizado,Target,Topic,prob_topic,outlier_iqr
0,asian exporter fear damage u.s.-japan rift mou...,0,0,0.006779,False
1,china daily say vermin eat 7 12 pct grain stoc...,0,0,0.011153,False
2,japan revise long term energy demand downwards...,0,0,0.009799,False
3,thai trade deficit widen quarter thailand trad...,0,0,0.009269,False
4,indonesia see cpo price rise sharply indonesia...,0,0,0.008703,False


In [22]:
## Reorganización del corpus

In [23]:
corpus['new_target']=0

In [24]:
corpus.loc[corpus.outlier_iqr==True,'new_target']=2
corpus.loc[corpus.Topic==1 &(corpus.outlier_iqr==False),'new_target']=1
corpus.loc[(corpus.Topic==0) & (corpus.outlier_iqr==False),'new_target']=0

In [27]:
corpus.new_target.unique()

array([0, 1, 2])

In [28]:
joblib.dump(corpus,'../data/processed/reuters/corpus_reordered.final.pck')

['../data/processed/reuters/corpus_reordered.final.pck']

In [29]:
corpus

,texto_normalizado,Target,Topic,prob_topic,outlier_iqr,new_target
0,asian exporter fear damage u.s.-japan rift mou...,0,0,0.006779,False,0
1,china daily say vermin eat 7 12 pct grain stoc...,0,0,0.011153,False,0
2,japan revise long term energy demand downwards...,0,0,0.009799,False,0
3,thai trade deficit widen quarter thailand trad...,0,0,0.009269,False,0
4,indonesia see cpo price rise sharply indonesia...,0,0,0.008703,False,0
...,...,...,...,...,...,...
10783,u.k money market shortage forecast revise bank...,0,0,0.015244,False,0
10784,knight ridder inc lt;krn > set quarterly qtly ...,0,1,0.986222,False,1
10785,technitrol inc lt;tnl > set quarterly qtly div...,0,1,0.986767,False,1
10786,nationwide cellular service inc lt;ncel > 4th ...,0,1,0.984818,False,1
